In [1]:
import os
os.environ [ "KMP_DUPLICATE_LIB_OK" ]= "TRUE"

%load_ext autoreload
%matplotlib notebook
%autoreload 2


# 0. what for
1. 用来检查data中究竟有没有domain shift的


# 1. Preparation

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from visdom import Visdom
from MLP.utils import setup_seed
import matplotlib.ticker as mticker

train_pct = 0.7
vali_pct = 0.2
test_pct = 0.1

target_data_path = "../data/targets_all"
data_key_path = "../data/target_datakey_all.csv"

##### Drawing Setting

# No type3
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = ['Helvetica']
plt.rcParams['font.size'] = 7
FIGSIZE = (4, 1)

## 1.2 read in

In [3]:
data_key = pd.read_csv(data_key_path)
target_all = os.listdir(target_data_path)

## 1.3 Functions: Draw and Get Data


In [4]:
def get_data_for_plot(train_lst,non_train_lst):
    duration_train = []
    duration_nontrain = []

    print(f"{len(train_lst),len(non_train_lst)}")
    for i in range(len(target_all)):
        data_file_path = os.path.join(target_data_path,target_all[i])
        data_i = pd.read_csv(data_file_path,encoding="utf-8")
        if i in train_lst:
            duration_train = duration_train+data_i.N.tolist()
        elif i in non_train_lst:
            duration_nontrain = duration_nontrain+data_i.N.tolist()
    return duration_train,duration_nontrain

In [5]:
# 读取所有seed下的数据
def get_data_for_plot_allseed(seed_lst):
    duration_train = []
    duration_nontrain = []
    DATA_len = len(target_all)

    for seed in seed_lst:
        shuffled_indices = np.random.permutation(DATA_len)
        train_lst = shuffled_indices[:int(opt.train_pct * DATA_len)]
        tmp = int((opt.train_pct + opt.vali_pct) * DATA_len)
        test_lst = shuffled_indices[int(opt.train_pct * DATA_len):tmp]
        # val_idx = shuffled_indices[tmp:]
        print(f"seed={seed},len of testing data={len(test_lst)}")

        for i in range(len(target_all)):
            data_file_path = os.path.join(target_data_path, target_all[i])
            data_i = pd.read_csv(data_file_path, encoding="utf-8")
            if i in train_lst:
                duration_train = duration_train + data_i.N.tolist()
            elif len(test_lst) == 0:  # 默认剩下的全部是testing set
                duration_nontrain = duration_nontrain + data_i.N.tolist()
            elif i in test_lst:  # Not all for testing
                duration_nontrain = duration_nontrain + data_i.N.tolist()

    return duration_train, duration_nontrain


In [6]:
# 画图
def draw_box_plot(train_lst, non_train_lst, title=None):

    duration_train,duration_nontrain = get_data_for_plot(train_lst)

    fig, ax = plt.subplots(figsize=FIGSIZE)

    # 绘制两个箱线图, 'edgecolor':'red',
    ax.boxplot([duration_train, duration_nontrain],showfliers=False,boxprops={'linewidth':4},medianprops={'color':'red','linewidth':2},whiskerprops={'linewidth':4},capprops={'linewidth':4})
    # 设置x轴标签
    ax.set_xticklabels(['Training set', 'Testing set'])
    # 设置y轴标签
    ax.set_ylabel('Durations')

    # plt.figure(figsize=(4))
    plt.title(title)

    # 显示图形
    plt.show()

# 2. 按照retail划分
- retail的hist

In [ ]:
data_key['retail'].hist()

In [ ]:
def split_dataset_retail(q = 0.8, VALI_TEST = False):

    retail_arr = data_key['retail'].sort_values(ignore_index=True)

    # 取前70%作为training
    retail_train = retail_arr.quantile(q=q)     # <= retail_train的属于training set
    print(f"when train_cnt={q}, the retail we split from is {retail_train}")
    train_lst = data_key[data_key['retail']<=retail_train].index.tolist()

    # 从non_train_lst中随意分裂出val和test
    non_train_lst = [i for i in range(0,len(target_all)) if i not in train_lst]
    # val_lst = (vali_pct/(vali_pct+test_pct))*len(non_train_lst)

    # 不返回vali和test的set
    if not VALI_TEST:
        return train_lst, non_train_lst
    else:
        return None

In [ ]:
train_lst, non_train_lst = split_dataset_retail(q=(0.9),VALI_TEST=False)
print(f"len of train set: {len(train_lst)}")
print(f"len of nontrain set: {len(non_train_lst)}")

In [ ]:
draw_box_plot(train_lst,non_train_lst,title = "Retail")

# 3. 按照bid inc划分

In [7]:
# 被select aside的bid inc
BIDINC = 0.01

In [8]:
# vali_pct = 0.1
# def split_dataset_bidinc(INC = 0.01, vali_pct = 0.1):
#
#     # 筛选出bidincrement!=INC的index
#     train_lst = [i for i in range(0,len(target_all)) if data_key.loc[i,'bidincrement']!=INC]
#
#     # 从non_train_lst中随意分裂出val和test
#     non_train_lst = [i for i in range(0,len(target_all)) if i not in train_lst]
#
#     tmp = int((vali_pct) * len(train_lst))
#     val_idx = train_lst[-tmp:]
#     train_idx = [i for i in train_lst if i not in val_idx]
#
#     return train_idx,non_train_lst


In [9]:

def split_dataset_bidinc(INC = 0.01, vali_pct = 0.1):

    # 筛选出bidincrement!=INC的index
    non_testing_lst = [i for i in range(0,len(target_all)) if data_key.loc[i,'bidincrement']!=INC]

    # 筛选出bidincrement==INC的index
    tesing_idx = [i for i in range(0,len(target_all)) if i not in non_testing_lst]

    # 取validation set
    tmp = int((vali_pct) * len(non_testing_lst))
    val_idx = non_testing_lst[-tmp:]

    # 取training set
    train_idx = [i for i in non_testing_lst if i not in val_idx]

    return train_idx,val_idx,tesing_idx


In [10]:
train_idx,val_idx,tesing_idx = split_dataset_bidinc(BIDINC,vali_pct = 0.1)
print(f"when BIDINC={BIDINC}, len of train set: {len(train_idx)}")
print(f"when BIDINC={BIDINC}, len of vali set: {len(val_idx)}")
print(f"when BIDINC={BIDINC}, len of testing set: {len(tesing_idx)}")

when BIDINC=0.01, len of train set: 968
when BIDINC=0.01, len of vali set: 107
when BIDINC=0.01, len of testing set: 201


In [15]:
draw_box_plot(train_lst,non_train_lst,title=f"BIDINC={BIDINC}")

TypeError: get_data_for_plot() missing 1 required positional argument: 'non_train_lst'

# 4. bid fee 划分

In [11]:
# 被select aside的bid fee
BIDFEE = 0.01

In [12]:
# def split_dataset_bidfee(FEE = 0.12,VALI_TEST = False):
#
#     # 筛选出bidinc!=FEE的index
#     train_lst = [i for i in range(0,len(target_all)) if data_key.loc[i,'bidfee']!=FEE]
#
#     # 从non_train_lst中随意分裂出val和test
#     non_train_lst = [i for i in range(0,len(target_all)) if i not in train_lst]
#     # val_lst = (vali_pct/(vali_pct+test_pct))*len(non_train_lst)
#
#     # 不返回vali和test的set
#     if not VALI_TEST:
#         return train_lst, non_train_lst
#     else:
#
#         tmp = int((vali_pct) * len(train_lst))
#         val_idx = train_lst[-tmp:]
#         train_idx = [i for i in train_lst if i not in val_idx]
#
#         return train_idx,non_train_lst


In [13]:
def split_dataset_bidfee(FEE = 0.01, vali_pct = 0.1):

    # 筛选出bidinc!=FEE的index
    non_testing_lst = [i for i in range(0,len(target_all)) if data_key.loc[i,'bidfee']!=FEE]

    # 筛选出bidinc==FEE的index
    tesing_idx = [i for i in range(0,len(target_all)) if i not in non_testing_lst]

    # 取validation set
    tmp = int((vali_pct) * len(non_testing_lst))
    val_idx = non_testing_lst[-tmp:]

    # 取training set
    train_idx = [i for i in non_testing_lst if i not in val_idx]

    return train_idx,val_idx,tesing_idx


In [14]:
train_idx,val_idx,tesing_idx = split_dataset_bidfee(BIDFEE,vali_pct = 0.1)
print(f"when BIDFEE={BIDFEE}, len of train set: {len(train_idx)}")
print(f"when BIDFEE={BIDFEE}, len of vali set: {len(val_idx)}")
print(f"when BIDFEE={BIDFEE}, len of testing set: {len(tesing_idx)}")

when BIDFEE=0.01, len of train set: 1077
when BIDFEE=0.01, len of vali set: 119
when BIDFEE=0.01, len of testing set: 80


In [ ]:
draw_box_plot(train_lst,non_train_lst,title=f"bid fee={BIDFEE}")


# 5. 按embedding划分
## 5.1 降维+聚类
1. embedding文件最后一列是desc

In [ ]:
LENTH=300
viz = Visdom()      # main()

prod_embedding_small_path = "../data/small_prod_embedding_"+str(LENTH)+".csv"
prod_embedding_large_path = "E:\DATA\large_dta\large_prod_embedding_"+str(LENTH)+".csv"

prod_embedding_small = pd.read_csv(prod_embedding_small_path,encoding="utf-8")
prod_embedding_large = pd.read_csv(prod_embedding_large_path,encoding="utf-8")

prod_embedding_all = pd.concat([prod_embedding_small,prod_embedding_large],axis=0,ignore_index=True)

In [ ]:
num_clusters = 7

In [ ]:
prod_embedding = prod_embedding_all.iloc[:,:LENTH]

# perplexity: 默认为30，数据集越大，需要参数值越大，建议值位5-50 , n_components=2 默认为2，嵌入空间的维度（嵌入空间的意思就是结果空间）
tsne_model = TSNE(perplexity=10, n_components=2, init='pca', n_iter=250, random_state=23)
new_values = tsne_model.fit_transform(prod_embedding)  # 新坐标
x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])


clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(prod_embedding)
cluster_assignment = clustering_model.labels_   # cluster 归属

# 长度为num_clusters的list，每个元素也是一个list
clustered_sentences = [[] for i in range(num_clusters)]

# cluster_assignment是array
# enumerate可以同时获得索引和值
# clustered_sentences记录了num_clusters个cluster里面的desc
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append( list(prod_embedding_all.loc[:,'desc'])[sentence_id] )


- 保存cluster分类

In [ ]:
# 以防万一，其实用不到
np.save("cluster_assignment",clustering_model.labels_)


In [ ]:
legend_str = [str(i) for i in range(num_clusters)]
viz.scatter(X=new_values, Y=cluster_assignment+1,
        opts=dict(
            colormap='Jet',
            markersymbol="dot",  # "x"
            markersize=10,
            legend = legend_str,
            textlabels= list(prod_embedding_all.loc[:,'desc'])
        ))


In [ ]:
cluster_prod_cluster_df = pd.DataFrame(clustering_model.labels_,columns=['cluster'])
cluster_prod_cluster_df.head()

In [ ]:
cluster_prod_cluster_df.groupby(['cluster']).size()

## 5.2 按照选定的cluster进行

In [ ]:
CLUSTER_CHOSEN = 6

In [ ]:
def split_dataset_emb(CLUSTER = 0,VALI_TEST = False):

    # 筛选出bidinc!=的index
    train_lst = [i for i in range(0,len(target_all)) if cluster_prod_cluster_df.loc[i,'cluster']!=CLUSTER]

    # 从non_train_lst中随意分裂出val和test
    non_train_lst = [i for i in range(0,len(target_all)) if i not in train_lst]
    # val_lst = (vali_pct/(vali_pct+test_pct))*len(non_train_lst)

    # 不返回vali和test的set
    if not VALI_TEST:
        return train_lst, non_train_lst
    else:
        return None


In [ ]:
train_lst, non_train_lst = split_dataset_emb(CLUSTER=CLUSTER_CHOSEN,VALI_TEST=False)
print(f"when CLUSTER_CHOSEN={CLUSTER_CHOSEN}, len of train set: {len(train_lst)}")
print(f"when CLUSTER_CHOSEN={CLUSTER_CHOSEN}, len of nontrain set: {len(non_train_lst)}")

In [ ]:
draw_box_plot(train_lst, non_train_lst, title=f"CLUSTER_CHOSEN={CLUSTER_CHOSEN}")

In [ ]:
cluster_df_path = "../data/cluster_assignment_N=" + str(num_clusters)+ "CHOSEN="+str(CLUSTER_CHOSEN)+".csv"
cluster_prod_cluster_df.to_csv(cluster_df_path,header=True,index=False,encoding="utf-8")

# 6. Plot in Paper
## 6.1 横向
### 6.1.1 inc+fee
1. 画到两个subplot中的bid inc和bid fee

In [15]:
# 被select aside的bid inc
BIDINC = 0.01
BIDFEE = 0.01
train_idx,val_idx,tesing_idx = split_dataset_bidinc(BIDINC,vali_pct = 0.1)
print(f"when BIDINC={BIDINC}, len of train set: {len(train_idx)}")
print(f"when BIDINC={BIDINC}, len of vali set: {len(val_idx)}")
print(f"when BIDINC={BIDINC}, len of testing set: {len(tesing_idx)}")
tesing_idx_inc = tesing_idx
nontesting_idx_inc = train_idx + val_idx

train_idx,val_idx,tesing_idx = split_dataset_bidfee(BIDFEE,vali_pct = 0.1)
print(f"when BIDFEE={BIDFEE}, len of train set: {len(train_idx)}")
print(f"when BIDFEE={BIDFEE}, len of vali set: {len(val_idx)}")
print(f"when BIDFEE={BIDFEE}, len of testing set: {len(tesing_idx)}")
tesing_idx_fee = tesing_idx
nontesting_idx_fee = train_idx + val_idx

when BIDINC=0.01, len of train set: 968
when BIDINC=0.01, len of vali set: 107
when BIDINC=0.01, len of testing set: 201
when BIDFEE=0.01, len of train set: 1077
when BIDFEE=0.01, len of vali set: 119
when BIDFEE=0.01, len of testing set: 80


In [16]:
duration_nontesting_inc,duration_tesing_inc = get_data_for_plot(nontesting_idx_inc,tesing_idx_inc)
duration_nontesting_fee,duration_tesing_fee = get_data_for_plot(nontesting_idx_fee,tesing_idx_fee)

(1075, 201)
(1196, 80)


In [17]:
len(duration_nontesting_fee),len(duration_tesing_fee)

(106959, 8872)

In [18]:
len(duration_nontesting_inc),len(duration_tesing_inc)

(95408, 20423)

In [54]:
labels = ['Testing Data','Training & Validation']
medianprops = {'color':"red",'linewidth':1}
whiskerprops = {'color':"#595959",'linewidth':1}
capprops={'color':"#595959",'linewidth':1}
boxprops={'color':"#595959",'linewidth':1}          # box style
# bbox_props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)       # title style
bbox_props = dict(boxstyle='square', linestyle='--', facecolor='white',edgecolor="#595959")
font_type = plt.rcParams['font.family']
print(font_type)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = ['Helvetica']


# Camera-ready
# x = 0.95  # Title position
# y= 0.3      # Title position
# fontsize=7
# fontsize_title=7
# fontdict={"size":fontsize}            # Titles
# fontdict_title={"size":fontsize_title}            # Titles
# matplotlib.rcParams['pdf.fonttype'] = 42  # Embedded fonts
# # plt.switch_backend('pgf')
# plt.rcParams['font.size'] = 7
# FIGSIZE = (3.5, 1.2)
# labelpad = 0.1
# widths = 0.5

# Thesis
x = 0.95    # Title position
y= 0.6      # Title position
fontsize=12
fontsize_title=12
fontdict={"size":fontsize}            # Titles
fontdict_title={"size":fontsize_title}            # Titles
matplotlib.rcParams['pdf.fonttype'] = 42  # Embedded fonts
# plt.switch_backend('pgf')
plt.rcParams['font.size'] = 12
FIGSIZE = (8, 2.2)
labelpad = 0.3
widths = 0.7


['sans-serif']


In [55]:
figs,axs = plt.subplots(2, 1,figsize=FIGSIZE,sharex=True) #

# Plot-1
axs[0].boxplot([duration_tesing_inc,duration_nontesting_inc],labels=labels,showfliers=False,vert=False,widths=widths,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

# 设置y轴
axs[0].set_yticklabels(labels,fontsize=fontsize)

# 设置标题
title0 = axs[0].set_title('Split by Bid Increment',fontdict=fontdict_title, loc="right",x=x,y=y)
title0.set_bbox(bbox_props)

# Plot-2
axs[1].boxplot([duration_tesing_fee,duration_nontesting_fee],labels=labels,showfliers=False,vert=False,widths=widths,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)


title1 = axs[1].set_title('Split by Bid Fee',fontdict=fontdict_title, loc="right",x=x,y=y)
title1.set_bbox(bbox_props)

axs[1].set_xlabel('Durations',fontdict=fontdict,labelpad = labelpad)

# get_xticks(): 返回 xaxis 的刻度位置。
# 必须使用FixedLocator定位器定位到散点图的x轴
# axs[1].get_xticks()  #若不传递任何参数，则返回当前刻度值

xlabels = axs[1].get_xticks().tolist()
axs[1].xaxis.set_major_locator(mticker.FixedLocator(xlabels))
axs[1].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()],fontdict=fontdict)

# axs[1].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()],fontdict=fontdict)   # 使用这个版本会有warning并且xlable不正确

# 设置x轴
axs[1].set_yticklabels(labels,fontsize=fontsize)

plt.gcf().subplots_adjust(bottom=0.3)

plt.tight_layout(h_pad=0.2)
# plt.subplots_adjust(top=0.15)


plt.savefig('zz.pdf')

# 显示图形
figs.show()

<IPython.core.display.Javascript object>

In [42]:
from MLP.Config.config_MDN import DefaultConfig
opt = DefaultConfig()

In [41]:
seed_lst = [3,31,204,223,407]

In [42]:
duration_train_seed,duration_nontrain_seed = get_data_for_plot_allseed(seed_lst)

seed=3,len of testing data=255
seed=31,len of testing data=255
seed=204,len of testing data=255
seed=223,len of testing data=255
seed=407,len of testing data=255


In [43]:
# 只画一组seed
figs,axs = plt.subplots(1, 1,figsize=(6,2)) #

# Plot-3
axs.boxplot([duration_nontrain_seed,duration_train_seed],labels=labels,showfliers=False,vert=False,widths=0.5,boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

# title1 = axs.set_title('ALL',fontdict=fontdict, loc="right",x=0.92, y=0.78)
# title1.set_bbox(bbox_props)
axs.set_xlabel('Durations',fontdict=fontdict)
# axs.set_xticklabels([f'{int(y/1000)}k' for y in axs.get_xticks()])
axs.set_yticklabels(labels,fontsize=fontsize)



# plt.gcf().subplots_adjust(bottom=0.15)
# 显示图形
plt.tight_layout()
figs.show()
plt.savefig('zz.pdf')


<IPython.core.display.Javascript object>

In [366]:
# 画3组
figs,axs = plt.subplots(3, 1,figsize=(6,4),sharex=True) #

# Plot-1
axs[0].boxplot([duration_nontrain_inc,duration_train_inc],labels=labels,showfliers=False,vert=False,widths=0.5,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

# 设置y轴
axs[0].set_yticklabels(labels,fontsize=fontsize)
# 设置标题
title0 = axs[0].set_title('Bid Increment',fontdict=fontdict, loc="right",x=x,y=y)
title0.set_bbox(bbox_props)

# Plot-2
axs[1].boxplot([duration_nontrain_fee,duration_train_fee],labels=labels,showfliers=False,vert=False,widths=0.5,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

title1 = axs[1].set_title('Bid Fee',fontdict=fontdict, loc="right",x=x,y=y)
title1.set_bbox(bbox_props)
# axs[1].set_xlabel('Durations',fontdict=fontdict)
axs[1].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()])
axs[1].set_yticklabels(labels,fontsize=fontsize)

# Plot-3
axs[2].boxplot([duration_nontrain_seed,duration_train_seed],labels=labels,showfliers=False,vert=False,widths=0.5,boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

title1 = axs[2].set_title('ALL',fontdict=fontdict, loc="right",x=x, y=y)
title1.set_bbox(bbox_props)
axs[2].set_xlabel('Durations',fontdict=fontdict)
axs[2].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()])
axs[2].set_yticklabels(labels,fontsize=fontsize)


# plt.tight_layout()
plt.gcf().subplots_adjust(bottom=0.15)
# 显示图形
figs.show()
plt.savefig('zz.pdf')


<IPython.core.display.Javascript object>

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6080\3268222175.py:20: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[1].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6080\3268222175.py:29: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[2].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()])


### 6.1.3 retail + emb

In [ ]:
train_lst_retail, non_train_lst_retail = split_dataset_retail(q=(0.87),VALI_TEST=False)
print(f"len of train set: {len(train_lst_retail)}")
print(f"len of nontrain set: {len(non_train_lst_retail)}")

In [275]:
# 读取csv得到lst of index
cluster_assignment_N = pd.read_csv("../data/cluster_assignment_N=7CHOSEN=2.csv", encoding="utf-8")
cluster_assignment_N.head()
train_lst_emb = cluster_assignment_N[cluster_assignment_N.cluster!=2].index.tolist()
non_train_lst_emb = cluster_assignment_N[cluster_assignment_N.cluster==2].index.tolist()
print(f"len of train set: {len(train_lst_emb)}")
print(f"len of nontrain set: {len(non_train_lst_retail)}")

len of train set: 1164
len of nontrain set: 165


In [276]:
duration_train_retail,duration_nontrain_retail = get_data_for_plot(train_lst_retail)
duration_train_emb,duration_nontrain_emb = get_data_for_plot(train_lst_emb)

In [277]:
labels = ['Testing\nData','Training\nData']
medianprops = {'color':"red",'linewidth':3}
whiskerprops = {'color':"#595959",'linewidth':3}
capprops={'color':"#595959",'linewidth':3}
boxprops={'color':"#595959",'linewidth':3}          # box style

# bbox_props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)       # title style
bbox_props = dict(boxstyle='square', linestyle='--', facecolor='white',edgecolor='black')
fontsize=12
fontdict={"size":fontsize}            # Titles and Lables

In [279]:
figs,axs = plt.subplots(2, 1,figsize=(6,3),sharex=True) #

# Plot-1
axs[0].boxplot([duration_nontrain_retail,duration_train_retail],labels=labels,showfliers=False,vert=False,widths=0.5,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

# 设置y轴
axs[0].set_yticklabels(labels,fontsize=fontsize)

# 设置标题
title0 = axs[0].set_title('Retail Price',fontdict=fontdict, loc="right",x=0.95,y=0.70)
title0.set_bbox(bbox_props)

# Plot-2
axs[1].boxplot([duration_nontrain_emb,duration_train_emb],labels=labels,showfliers=False,vert=False,widths=0.5,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

# 设置x轴标签
title1 = axs[1].set_title('Product Embedding',fontdict=fontdict, loc="right",x=0.95,y=0.70)
title1.set_bbox(bbox_props)

axs[1].set_xlabel('Durations',fontdict=fontdict)

axs[1].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()])

# 设置x轴
axs[1].set_yticklabels(labels,fontsize=fontsize)

# plt.tight_layout()
plt.gcf().subplots_adjust(bottom=0.15)
# 显示图形
figs.show()

plt.savefig('zz.pdf')


<IPython.core.display.Javascript object>

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6080\1523935958.py:24: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[1].set_xticklabels([f'{int(y/1000)}k' for y in axs[1].get_xticks()])


## 6.2 纵向

In [245]:
figs,axs = plt.subplots(1, 2,figsize=(5,4))

# Plot-1
axs[0].boxplot([duration_train_inc, duration_nontrain_inc],labels=labels,showfliers=False,widths=0.5,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

# 设置x轴
axs[0].set_ylabel('Durations',fontdict=fontdict)

axs[0].set_yticklabels([f'{int(y/1000)}k' for y in axs[0].get_yticks()])

# 设置y轴

axs[0].set_xticklabels(labels,fontsize=fontsize)

# 设置标题
title0 = axs[0].set_title('Bid Increment',fontdict=fontdict)
# title0.set_bbox(bbox_props)

# Plot-2
axs[1].boxplot([duration_train_fee, duration_nontrain_fee],labels=labels,showfliers=False,widths=0.5,
                boxprops=boxprops,medianprops=medianprops,whiskerprops=whiskerprops,capprops = capprops)

# 设置x轴标签
title1 = axs[1].set_title('Bid Fee',fontdict=fontdict)
# title1.set_bbox(bbox_props)


axs[1].set_yticklabels([f'{int(y/1000)}k' for y in axs[1].get_yticks()])

# 设置x轴
axs[1].set_xticklabels(labels,fontsize=fontsize)


# 显示图形
figs.show()

# plt.savefig('zz.png')
plt.savefig('zz.pdf')


<IPython.core.display.Javascript object>

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6080\383081471.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[0].set_yticklabels([f'{int(y/1000)}k' for y in axs[0].get_yticks()])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6080\383081471.py:29: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[1].set_yticklabels([f'{int(y/1000)}k' for y in axs[1].get_yticks()])


# 7. Domain check for current testing and training data

In [391]:
from MLP.mydataset import *
from MLP.Config.config_MDN import DefaultConfig
opt = DefaultConfig()

- 得到所有seed下的data

In [392]:
seed_lst = [3,31,204,223,407]

In [405]:
def get_data_for_plot_allseed(seed_lst):
    duration_train = []
    duration_nontrain = []
    DATA_len = len(target_all)


    for seed in seed_lst:
        setup_seed(seed)
        shuffled_indices = np.random.permutation(DATA_len)
        train_lst = shuffled_indices[:int(opt.train_pct * DATA_len)]
        tmp = int((opt.train_pct + opt.vali_pct) * DATA_len)
        test_lst = shuffled_indices[int(opt.train_pct * DATA_len):tmp]
        # val_idx = shuffled_indices[tmp:]
        print(f"seed={seed}")
        # print(f"seed={seed},len of testing data={len(test_lst)}")
        print(test_lst)
        for i in range(len(target_all)):
            data_file_path = os.path.join(target_data_path, target_all[i])
            data_i = pd.read_csv(data_file_path, encoding="utf-8")
            if i in train_lst:
                duration_train = duration_train + data_i.N.tolist()
            elif i in test_lst:  # For testing
            # else:
                duration_nontrain = duration_nontrain + data_i.N.tolist()
        print(f"seed = {seed}, len of testing set = {len(duration_nontrain)}")
    return duration_train, duration_nontrain

In [406]:
duration_train_seed,duration_nontrain_seed = get_data_for_plot_allseed(seed_lst)

seed=3
[ 801  265  535  919  451  990  914  404 1175  490 1138  541  693  739
  364  629  459  104  833  268  608 1181 1244  904 1109  661  241  430
  814 1047  203 1234  595  630   96  467  975  492  166 1018  701  694
    9  604  878  315  773  361  138  823  276  384  568 1134  772  922
   88   90  765  280  882  651 1255  460  992  799  300 1049  202  402
  340  365   23  345  676 1235  334  310 1213  640  353  735 1246  240
  666  318 1093  262  940   98  167  496  725  275   89  714  515 1063
  331  219  433 1222 1121  486  658 1020  531  840 1099  260  956  810
  305  876 1067  702  771 1204  196  140  594  122  494  821  110   84
 1095  513  691  482 1194  369  793  926  273  396 1271  797  961  418
  375  463  470  559  302  582  798   11 1068  538  627  281  983 1264
  223  672  966 1263  994  580 1045   77  547 1272  748  826  125  757
  576  621  767 1269 1030 1152  802  844 1074 1254 1094  259  282  634
  474  455  320  434 1230  299  436 1162  438  325  403  564  292  278

In [407]:
fig, ax = plt.subplots(1,1)
# 绘制两个箱线图, 'edgecolor':'red',
ax.boxplot([duration_train_seed, duration_nontrain_seed],showfliers=False,boxprops={'linewidth':4},medianprops={'color':'red','linewidth':2},whiskerprops={'linewidth':4},capprops={'linewidth':4})
# 设置x轴标签
ax.set_xticklabels(['Training set', 'Testing set'])
# 设置y轴标签
ax.set_ylabel('Durations')

# plt.figure(figsize=(4,4))
title = "All"
plt.title(title)

# 显示图形
plt.show()

<IPython.core.display.Javascript object>

In [350]:
plt.savefig('zz.pdf')
